In [1]:
import csv
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [2]:
file_repl = []
with open('checkins.dat', 'r') as f:
    file = f.readlines()
    for line in file:
        file_repl.append(line.strip().split('|'))
    for i in range(len(file_repl)):
        for j in range(len(file_repl[i])):
            file_repl[i][j] = file_repl[i][j].strip()
del file_repl[1]

In [3]:
with open('checkins.csv', 'w') as csv_file:
    wr = csv.writer(csv_file,delimiter=',')
    wr.writerows(file_repl)

In [4]:
df = pd.read_csv('checkins.csv', index_col=0)
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,venue_id,latitude,longitude,created_at
id,,,,,
984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [5]:
df.dropna(axis=0, inplace=True)
df.head()

,user_id,venue_id,latitude,longitude,created_at
id,,,,,
984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396634 entries, 984222 to 956733
Data columns (total 5 columns):
user_id       396634 non-null float64
venue_id      396634 non-null float64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(4), object(1)
memory usage: 15.1+ MB


In [7]:
df = df[:100000][['latitude','longitude']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 984222 to 127284
Data columns (total 2 columns):
latitude     100000 non-null float64
longitude    100000 non-null float64
dtypes: float64(2)
memory usage: 1.9+ MB


In [8]:
#%%time
clust = MeanShift(bandwidth=0.1).fit(df)
labels = clust.labels_
cluster_centers = clust.cluster_centers_
n_clusters_ = len(np.unique(labels))
print(n_clusters_)
print(cluster_centers[2])

3231
[  33.44638027 -111.90188756]


In [9]:
df['cl_label'] = labels
num_items = list(map(lambda x: df['cl_label'].value_counts()[x], labels))
df['cl_label_nums'] = num_items

In [10]:
df = df[df['cl_label_nums']>15]
df.head()

,latitude,longitude,cl_label,cl_label_nums
id,,,,
984222,38.895112,-77.036366,5,2409
984234,33.800745,-84.410520,7,1601
984291,45.523452,-122.676207,30,594
984318,40.764462,-111.904565,65,186
984232,33.448377,-112.074037,1,4692


In [34]:
def dist(a,b):
    return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5

def min_dist(a,b):
    d = []
    for i in range(len(b)):
        d.append(dist(a,b[i]))
    z = list(zip(d,b))
    return sorted(z,key=lambda tup: tup[0])[0][0], sorted(z,key=lambda tup: tup[0])[0][1]

In [35]:
cl_cent = list(zip(df['latitude'],df['longitude']))
distances = []
offices = []
office_coordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

for i in cl_cent:
    d, of = min_dist(i, office_coordinates)
    distances.append(d)
    offices.append(of)

df['min_dist'] = distances
df['nearest_office'] = offices
df.head()

,latitude,longitude,cl_label,cl_label_nums,min_dist,nearest_office
id,,,,,,
984222,38.895112,-77.036366,5,2409,13.435841,"(25.867736, -80.324116)"
984234,33.800745,-84.410520,7,1601,8.923639,"(25.867736, -80.324116)"
984291,45.523452,-122.676207,30,594,12.598470,"(33.751277, -118.18874)"
984318,40.764462,-111.904565,65,186,9.416773,"(33.751277, -118.18874)"
984232,33.448377,-112.074037,1,4692,6.122200,"(33.751277, -118.18874)"


In [43]:
df_sorted = df.sort_values(by=['min_dist'])[0:20]
df_sorted.sort_values(by=['nearest_office']).head()

,latitude,longitude,cl_label,cl_label_nums,min_dist,nearest_office
id,,,,,,
1019750,-33.873651,151.206890,420,28,0.005483,"(-33.868457, 151.205134)"
1008964,25.865096,-80.324496,419,28,0.002667,"(25.867736, -80.324116)"
615071,52.373056,4.892222,370,31,0.009222,"(52.378894, 4.885084)"
222112,52.373056,4.892222,370,31,0.009222,"(52.378894, 4.885084)"
758619,52.373056,4.892222,370,31,0.009222,"(52.378894, 4.885084)"
